# LightGBM: ARI

### Setup

In [1]:
import sys
sys.path.append('../')
from src.load_data import *
from src.realtime_utils import *
from src.hp_tuning_functions import *

In [2]:
from darts.models.forecasting.lgbm import LightGBMModel

/home/wolffram/.conda/envs/RESPINOW-ML/lib/python3.9/site-packages/statsforecast/utils.py:231: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),


In [3]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dwolffram (dwolffram-karlsruhe-institute-of-technology). Use `wandb login --relogin` to force relogin


True

# Load best model

In [4]:
params = get_best_parameters('sari-lightgbm', 'lightgbm', 'WIS')

WIS of best run: 447.06267950493736


In [5]:
{key: params[key] for key in sorted(params)}

{'colsample_bytree': 0.8,
 'lags': 8,
 'lags_future_covariates': [0, 1],
 'lags_past_covariates': {'agi-are-DE': 8,
  'default_lags': 1,
  'agi-are-00-04': 8,
  'agi-are-05-14': 8,
  'agi-are-60+': 8,
  'agi-are-35-59': 8,
  'agi-are-15-34': 8},
 'learning_rate': 0.1,
 'max_bin': 1024,
 'max_depth': -1,
 'min_child_samples': 10,
 'min_split_gain': 0,
 'model': 'lightgbm',
 'multiple_series': False,
 'n_estimators': 500,
 'num_leaves': 31,
 'reg_alpha': 0.5,
 'reg_lambda': 0.5,
 'sample_weight': 'linear',
 'subsample': 0.8,
 'subsample_freq': 1,
 'use_covariates': True,
 'use_encoders': True,
 'use_features': False,
 'use_static_covariates': False}

In [6]:
name = params.pop('model')
multiple_series = params.pop('multiple_series')
use_covariates = params.pop('use_covariates')
use_features = params.pop('use_features')
use_encoders = params.pop('use_encoders')
sample_weight = params.pop('sample_weight')

if not use_covariates:
    del params['lags_past_covariates']
    
model = LightGBMModel(
    **params,
    output_chunk_length=HORIZON,
    add_encoders=ENCODERS if use_encoders else None,
    likelihood="quantile",
    quantiles=QUANTILES,
    verbose=-1,
)

# Train model

In [ ]:
for forecast_date in FORECAST_DATES:
    
    path = f'../models/post-covid/{forecast_date}/'
    os.makedirs(path, exist_ok=True)

    targets, covariates = load_realtime_training_data(as_of=forecast_date)

    for seed in RANDOM_SEEDS:
        model_path = path + f'{forecast_date}-lightgbm-{seed}.pt'
        print(model_path)
        model = LightGBMModel(
            **params,
            output_chunk_length=HORIZON,
            add_encoders=ENCODERS if use_encoders else None,
            likelihood="quantile",
            quantiles=QUANTILES,
            verbose=-1,
            random_state=seed
        )
        model.fit(targets, past_covariates=covariates if use_covariates else None, sample_weight=sample_weight)
        model.save(model_path)

../models/post-covid/2023-11-16/2023-11-16-lightgbm-1.pt
../models/post-covid/2023-11-16/2023-11-16-lightgbm-2.pt
../models/post-covid/2023-11-16/2023-11-16-lightgbm-3.pt
../models/post-covid/2023-11-16/2023-11-16-lightgbm-4.pt
../models/post-covid/2023-11-16/2023-11-16-lightgbm-5.pt


# Forecast

In [8]:
targets, covariates = load_realtime_training_data()

In [9]:
def compute_ensemble(forecast_date, model_name, export=False):
    dfs = []
    for seed in RANDOM_SEEDS:
        print(seed)
        model_path = f'../models/post-covid/{forecast_date}/{forecast_date}-{model_name}-{seed}.pt'
        model = LightGBMModel.load(model_path)
        df = compute_forecast(model, targets, covariates, forecast_date, HORIZON, NUM_SAMPLES, vincentization=False, probabilistic_nowcast=True, local=True)
        dfs.append(df)

    df = pd.concat(dfs)
    df = df.groupby(['location', 'age_group', 'forecast_date', 'target_end_date', 'horizon', 'type', 'quantile']).agg({'value': 'mean'}).reset_index()

    df = df.sort_values(['location', 'age_group', 'horizon', 'quantile'])
    
    if export:
        df.to_csv(f'../data/post-covid/submissions/lightgbm_new/{forecast_date}-icosari-sari-lightgbm_new.csv', index=False)

    return df

In [10]:
forecasts = []
for forecast_date in FORECAST_DATES:
    print(forecast_date)
    forecast = compute_ensemble(forecast_date, 'lightgbm', export=True)
    forecasts.append(forecast)

2023-11-16
1
2
3
4
5
6
7
8
9
10
2023-11-23
1
2
3
4
5
6
7
8
9
10
2023-11-30
1
2
3
4
5
6
7
8
9
10
2023-12-07
1
2
3
4
5
6
7
8
9
10
2023-12-14
1
2
3
4
5
6
7
8
9
10
2023-12-21
1
2
3
4
5
6
7
8
9
10
2023-12-28
1
2
3
4
5
6
7
8
9
10
2024-01-04
1
2
3
4
5
6
7
8
9
10
2024-01-11
1
2
3
4
5
6
7
8
9
10
2024-01-18
1
2
3
4
5
6
7
8
9
10
2024-01-25
1
2
3
4
5
6
7
8
9
10
2024-02-01
1
2
3
4
5
6
7
8
9
10
2024-02-08
1
2
3
4
5
6
7
8
9
10
2024-02-15
1
2
3
4
5
6
7
8
9
10
2024-02-22
1
2
3
4
5
6
7
8
9
10
2024-02-29
1
2
3
4
5
6
7
8
9
10
2024-03-07
1
2
3
4
5
6
7
8
9
10
2024-03-14
1
2
3
4
5
6
7
8
9
10
2024-03-21
1
2
3
4
5
6
7
8
9
10
2024-03-28
1
2
3
4
5
6
7
8
9
10
2024-04-04
1
2
3
4
5
6
7
8
9
10
2024-04-11
1
2
3
4
5
6
7
8
9
10
2024-04-18
1
2
3
4
5
6
7
8
9
10
2024-04-25
1
2
3
4
5
6
7
8
9
10
2024-05-02
1
2
3
4
5
6
7
8
9
10
2024-05-09
1
2
3
4
5
6
7
8
9
10
2024-05-16
1
2
3
4
5
6
7
8
9
10
2024-05-23
1
2
3
4
5
6
7
8
9
10
2024-05-30
1
2
3
4
5
6
7
8
9
10
2024-06-06
1
2
3
4
5
6
7
8
9
10
2024-06-13
1
2
3
4
5
6
7
8
9
10
2024-06-